In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import glob
import os
#pd.set_option('display.max_rows', 200)

In [10]:
data = pd.read_excel('..\\Datos Partidos\\Arsenal_Situaciones.xlsx')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Session Start Date   26 non-null     datetime64[ns]
 1   Event                26 non-null     object        
 2   Session Name         26 non-null     object        
 3   Session Start        26 non-null     object        
 4   Session End          26 non-null     object        
 5   Tag Description      26 non-null     object        
 6   Tag Notes            0 non-null      float64       
 7   Tag Start            26 non-null     object        
 8   Tag End              26 non-null     object        
 9   Tag Duration (secs)  26 non-null     float64       
 10  Categoria            26 non-null     object        
 11  Zona2                26 non-null     float64       
 12  Zona                 26 non-null     float64       
 13  Accion               26 non-null     

In [11]:
data.columns = ['Fecha','etiquetado','name','Session Start','Session End','Evento','Tag Notes','Tag Start','Tag End','Tag Duration (secs)','Categoria','Zona inicial','Zona Final','Accion','Jugada']

In [12]:
data.sample(5)

,Fecha,etiquetado,name,Session Start,Session End,Evento,Tag Notes,Tag Start,Tag End,Tag Duration (secs),Categoria,Zona inicial,Zona Final,Accion,Jugada
2,2022-07-02,Arsenal,Situaciones,15:34:01,17:36:19,A FAVOR,NaN,00:24:11,00:24:13,2000.0,PELOTA PARADA,9.0,9.0,VOLVER,2.0
4,2022-07-02,Arsenal,Situaciones,15:34:01,17:36:19,A FAVOR,NaN,00:24:22,00:24:24,2000.0,PELOTA PARADA,8.2,8.2,REMATE AEREO,2.0
10,2022-07-02,Arsenal,Situaciones,15:34:01,17:36:19,A FAVOR,NaN,01:24:56,01:24:58,2000.0,ATAQUE COMBINADO,8.2,8.1,REMATE AFUERA,5.0
16,2022-07-02,Arsenal,Situaciones,15:34:01,17:36:19,A FAVOR,NaN,01:28:10,01:28:12,2000.0,ATAQUE COMBINADO,9.0,8.2,VOLVER,6.0
19,2022-07-02,Arsenal,Situaciones,15:34:01,17:36:19,A FAVOR,NaN,01:28:42,01:28:44,2000.0,PELOTA PARADA,8.2,8.2,REMATE AEREO,7.0


In [13]:
'''CORREGIR MAYUSCULAS Y COSAS DE EVENTOS'''

data['Evento'] = data['Evento'].str.lower().str.strip()
data['Categoria'] = data['Categoria'].str.lower().str.strip()
data['Accion'] = data['Accion'].str.lower().str.strip()

## Estadisticas 

In [14]:
data.groupby(['Evento','Categoria'])['Jugada'].nunique().to_frame('Cantidad')

Cantidad
Evento    Categoria                 
a favor   ataque combinado         2
          pelota parada            2
          transicion               1
en contra pelota parada            3
          transicion               1

### Armo las coordenadas

In [15]:
data['Zona inicial'] = data['Zona inicial'].replace('2A',2.1,regex=True)
data['Zona inicial'] = data['Zona inicial'].replace('2B',2.2,regex=True)
data['Zona inicial'] = data['Zona inicial'].replace('8A',8.1,regex=True)
data['Zona inicial'] = data['Zona inicial'].replace('8B',8.2,regex=True)
data['Zona Final'] = data['Zona Final'].replace('2A',2.1,regex=True)
data['Zona Final'] = data['Zona Final'].replace('2B',2.2,regex=True)
data['Zona Final'] = data['Zona Final'].replace('8A',8.1,regex=True)
data['Zona Final'] = data['Zona Final'].replace('8B',8.2,regex=True)

In [16]:
data['Zona inicial'].value_counts()

8.2    9
2.2    4
5.0    3
3.0    2
9.0    2
2.1    2
8.1    2
4.0    1
7.0    1
Name: Zona inicial, dtype: int64

In [17]:
data['Zona Final'].value_counts()

6.0    4
9.0    4
8.2    4
5.0    3
3.0    2
1.0    2
2.1    2
8.1    2
4.0    2
7.0    1
Name: Zona Final, dtype: int64

In [18]:
map_zona_coord_x = {  
1:45,
2.1:70,
2.2:30,
3:45,
4:136,
5:136,
6:136,
7:227,
8.1:255,
8.2:207,
9:227
}

In [19]:
map_zona_coord_y = {  
1:135,
2.1:81,
2.2:81,
3:27,
4:135,
5:81,
6:27,
7:135,
8.1:81,
8.2:81,
9:81
}

In [20]:
data['coord.final.num'] = np.where(data['Zona Final'] == 'volver',None, data['Zona Final']).astype('float')


In [21]:
data['Zona inicial x'] = data['Zona inicial'].map(map_zona_coord_x)
data['Zona Final x'] = data['coord.final.num'].map(map_zona_coord_x)
data['Zona inicial y'] = data['Zona inicial'].map(map_zona_coord_y)
data['Zona Final y'] = data['coord.final.num'].map(map_zona_coord_y)
data.iloc[:,-5:]

,coord.final.num,Zona inicial x,Zona Final x,Zona inicial y,Zona Final y
0,6.0,45,136,27,27
1,3.0,30,45,81,27
2,9.0,227,227,81,81
3,9.0,207,227,81,81
4,8.2,207,207,81,81
5,1.0,30,45,81,135
6,1.0,70,45,81,135
7,2.1,30,70,81,81
8,6.0,136,136,81,27
9,5.0,255,136,81,81


In [22]:
# pasar en ambos formatos. Crear las dos cols originales q sean strings

data['coord.inicial'] = round(data['Zona inicial x'],0).astype('str').str.split('.').str[0]+','+round(data['Zona inicial y'],0).astype('str').str.split('.').str[0]
data['coord.final'] = round(data['Zona Final x'],0).astype('str').str.split('.').str[0]+','+round(data['Zona Final y'],0).astype('str').str.split('.').str[0]


In [23]:
data['coord.inicial'] = np.where(data['coord.inicial']=='nan,nan', None, data['coord.inicial'])
data['coord.final'] = np.where(data['coord.final']=='nan,nan', None, data['coord.final'])

In [24]:
data[['coord.inicial','coord.final']]

,coord.inicial,coord.final
0,"45,27","136,27"
1,"30,81","45,27"
2,"227,81","227,81"
3,"207,81","227,81"
4,"207,81","207,81"
5,"30,81","45,135"
6,"70,81","45,135"
7,"30,81","70,81"
8,"136,81","136,27"
9,"255,81","136,81"


In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Fecha                26 non-null     datetime64[ns]
 1   etiquetado           26 non-null     object        
 2   name                 26 non-null     object        
 3   Session Start        26 non-null     object        
 4   Session End          26 non-null     object        
 5   Evento               26 non-null     object        
 6   Tag Notes            0 non-null      float64       
 7   Tag Start            26 non-null     object        
 8   Tag End              26 non-null     object        
 9   Tag Duration (secs)  26 non-null     float64       
 10  Categoria            26 non-null     object        
 11  Zona inicial         26 non-null     float64       
 12  Zona Final           26 non-null     float64       
 13  Accion               26 non-null     

In [26]:
data['Temporada'] = 'Temp 02-2022'
data['partido'] = 'Arsenal'

In [ ]:
data.to_excel("../Datos generados/reserva02-22-fecha6_arsenal_situaciones.xlsx",index=False)